### General Security Analysis for Face Template Protection Methods from Cryptographic Hash Functions

- Seunghun Paik, Minsu Kim, Sunpill Kim, and Jae Hong Seo

- Department of Mathematics & Research Institute for Natural Sciences, Hanyang University

- Submitted to IEEE Transactions on Information Forensics and Security.

#### This notebook measures the upper bound of TAR for CtH-based BTPs.

In [ ]:
### Configs for models & datasets
from config import cfg
device = "cuda:1"
batch_size = 256

#### Step 1. Load Benchmark Datasets

In [ ]:
import os
from feat_tools import load_bin

lfw_dir = os.path.join(cfg.img_dir, "lfw.bin")
cfp_dir = os.path.join(cfg.img_dir, "cfp_fp.bin")
age_dir = os.path.join(cfg.img_dir, "agedb_30.bin")

lfw = load_bin(lfw_dir, (112,112))
cfp = load_bin(cfp_dir, (112,112))
age = load_bin(age_dir, (112,112))

#### Step 3. Do Benchmarks for deriving the upper TAR

In [ ]:
### Setup Tables for security levels & thresholds
from estimator import make_table, plot_table

sec_levels = [128, 192, 256]
dims = [1<<i for i in range(7, 13)]
table = make_table(sec_levels, dims)

# This draws Fig. 2.
plot_table(table, sec_levels)

##### InsightFace

In [ ]:
from benchmark import end_to_end_benchmark
from backbones import get_backbone

In [ ]:
print("Model: ", cfg.model1[-1])
model = get_backbone(cfg.model1, device)
end_to_end_benchmark(lfw, model, batch_size, device, table, target_far = 0, suffix = r"$I_{1}$")
end_to_end_benchmark(cfp, model, batch_size, device, table, target_far = 0, suffix = r"$I_{1}$")
end_to_end_benchmark(age, model, batch_size, device, table, target_far = 0, suffix = r"$I_{1}$")

In [ ]:
print("Model: ", cfg.model2[-1])
model = get_backbone(cfg.model2, device)
end_to_end_benchmark(lfw, model, batch_size, device, table, target_far = 0, suffix = r"$I_{2}$")
end_to_end_benchmark(cfp, model, batch_size, device, table, target_far = 0, suffix = r"$I_{2}$")
end_to_end_benchmark(age, model, batch_size, device, table, target_far = 0, suffix = r"$I_{2}$")

In [ ]:
print("Model: ", cfg.model3[-1])
model = get_backbone(cfg.model3, device)
end_to_end_benchmark(lfw, model, batch_size, device, table, target_far = 0, suffix = r"$I_{3}$")
end_to_end_benchmark(cfp, model, batch_size, device, table, target_far = 0, suffix = r"$I_{3}$")
end_to_end_benchmark(age, model, batch_size, device, table, target_far = 0, suffix = r"$I_{3}$")

##### OpenSphere

In [ ]:
print("Model: ", cfg.model4[-1])
model = get_backbone(cfg.model4, device)
end_to_end_benchmark(lfw, model, batch_size, device, table, target_far = 0, suffix = r"$O_{1}$")
end_to_end_benchmark(cfp, model, batch_size, device, table, target_far = 0, suffix = r"$O_{1}$")
end_to_end_benchmark(age, model, batch_size, device, table, target_far = 0, suffix = r"$O_{1}$")

In [ ]:
print("Model: ", cfg.model5[-1])
model = get_backbone(cfg.model5, device)
end_to_end_benchmark(lfw, model, batch_size, device, table, target_far = 0, suffix = r"$O_{2}$")
end_to_end_benchmark(cfp, model, batch_size, device, table, target_far = 0, suffix = r"$O_{2}$")
end_to_end_benchmark(age, model, batch_size, device, table, target_far = 0, suffix = r"$O_{2}$")

In [ ]:
print("Model: ", cfg.model6[-1])
model = get_backbone(cfg.model6, device)
end_to_end_benchmark(lfw, model, batch_size, device, table, target_far = 0, suffix = r"$O_{3}$")
end_to_end_benchmark(cfp, model, batch_size, device, table, target_far = 0, suffix = r"$O_{3}$")
end_to_end_benchmark(age, model, batch_size, device, table, target_far = 0, suffix = r"$O_{3}$")

##### CVLFace

In [ ]:
print("Model: ", cfg.model7[-1])
model = get_backbone(cfg.model7, device)
end_to_end_benchmark(lfw, model, batch_size, device, table, target_far = 0, suffix = r"$C_{1}$")
end_to_end_benchmark(cfp, model, batch_size, device, table, target_far = 0, suffix = r"$C_{1}$")
end_to_end_benchmark(age, model, batch_size, device, table, target_far = 0, suffix = r"$C_{1}$")

In [ ]:
print("Model: ", cfg.model8[-1])
model = get_backbone(cfg.model8, device)
end_to_end_benchmark(lfw, model, batch_size, device, table, target_far = 0, suffix = r"$C_{2}$")
end_to_end_benchmark(cfp, model, batch_size, device, table, target_far = 0, suffix = r"$C_{2}$")
end_to_end_benchmark(age, model, batch_size, device, table, target_far = 0, suffix = r"$C_{2}$")

In [ ]:
print("Model: ", cfg.model9[-1])

# Making Use of KPRPE is quite intricated...
# The following codes are based on the instruction from CVLFace 
# For more information, please refer to
# https://huggingface.co/minchul/cvlface_adaface_vit_base_kprpe_webface4m

from transformers import AutoModel
from huggingface_hub import hf_hub_download
import shutil
import os
import torch


# helpfer function to download huggingface repo and use model
def download(repo_id, path, HF_TOKEN=None):
    os.makedirs(path, exist_ok=True)
    files_path = os.path.join(path, 'files.txt')
    if not os.path.exists(files_path):
        hf_hub_download(repo_id, 'files.txt', token=HF_TOKEN, local_dir=path, local_dir_use_symlinks=False)
    with open(os.path.join(path, 'files.txt'), 'r') as f:
        files = f.read().split('\n')
    for file in [f for f in files if f] + ['config.json', 'wrapper.py', 'model.safetensors']:
        full_path = os.path.join(path, file)
        if not os.path.exists(full_path):
            hf_hub_download(repo_id, file, token=HF_TOKEN, local_dir=path, local_dir_use_symlinks=False)

            
# helpfer function to download huggingface repo and use model
def load_model_from_local_path(path, HF_TOKEN=None):
    cwd = os.getcwd()
    os.chdir(path)
    model = AutoModel.from_pretrained(path, trust_remote_code=True, token=HF_TOKEN)
    os.chdir(cwd)
    return model


# helpfer function to download huggingface repo and use model
def load_model_by_repo_id(repo_id, save_path, HF_TOKEN=None, force_download=False):
    if force_download:
        if os.path.exists(save_path):
            shutil.rmtree(save_path)
    download(repo_id, save_path, HF_TOKEN)
    return load_model_from_local_path(save_path, HF_TOKEN)


HF_TOKEN = 'YOUR HUGGINGFACE TOKEN'
path = os.path.expanduser('~/.cvlface_cache/minchul/cvlface_adaface_vit_base_kprpe_webface4m')
repo_id = 'minchul/cvlface_adaface_vit_base_kprpe_webface4m'
model = load_model_by_repo_id(repo_id, path, HF_TOKEN)

# input is a rgb image normalized.
from torchvision.transforms import Compose, ToTensor, Normalize, Resize
from PIL import Image
input = torch.randn((1,3,112,112))

# KPRPE also takes keypoints locations as input
path = os.path.expanduser('~/.cvlface_cache/minchul/cvlface_DFA_mobilenet')
aligner = load_model_by_repo_id('minchul/cvlface_DFA_mobilenet', path, HF_TOKEN)
aligned_x, orig_ldmks, aligned_ldmks, score, thetas, bbox = aligner(input)
keypoints = orig_ldmks  # torch.randn(1, 5, 2)
out = model(input, keypoints)

from torch import nn

class CVLWrapper(nn.Module):
    def __init__(self, aligner, model):
        super().__init__()
        self.aligner = aligner
        self.model = model
        
    def forward(self, x):
        _, orig_ldmks, _, _, _, _ = self.aligner(x)
        out = self.model(x, orig_ldmks)
        return out
    
model = CVLWrapper(aligner, model)
model = model.eval().to(device)

end_to_end_benchmark(lfw, model, batch_size, device, table, target_far = 0, suffix = r"$C_{3}$")
end_to_end_benchmark(cfp, model, batch_size, device, table, target_far = 0, suffix = r"$C_{3}$")
end_to_end_benchmark(age, model, batch_size, device, table, target_far = 0, suffix = r"$C_{3}$")

##### Miscellaneous

In [ ]:
print("Model: ", cfg.model10[-1])
model = get_backbone(cfg.model10, device)
end_to_end_benchmark(lfw, model, 256, device, table, target_far = 0, suffix = r"$M_{1}$")
end_to_end_benchmark(cfp, model, 256, device, table, target_far = 0, suffix = r"$M_{1}$")
end_to_end_benchmark(age, model, 256, device, table, target_far = 0, suffix = r"$M_{1}$")

In [ ]:
print("Model: ", cfg.model11[-1])
model = get_backbone(cfg.model11, device)
end_to_end_benchmark(lfw, model, 256, device, table, target_far = 0, suffix = r"$M_{2}$")
end_to_end_benchmark(cfp, model, 256, device, table, target_far = 0, suffix = r"$M_{2}$")
end_to_end_benchmark(age, model, 256, device, table, target_far = 0, suffix = r"$M_{2}$")

In [ ]:
print("Model: ", cfg.model12[-1])
model = get_backbone(cfg.model12, device)
end_to_end_benchmark(lfw, model, 256, device, table, target_far = 0, suffix = r"$M_{3}$")
end_to_end_benchmark(cfp, model, 256, device, table, target_far = 0, suffix = r"$M_{3}$")
end_to_end_benchmark(age, model, 256, device, table, target_far = 0, suffix = r"$M_{3}$")